# 训练量子隐形传态协议

<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## 概述

量子隐形传态（Quantum Teleportation）是可以通过本地操作和经典通信（LOCC）协议完成的另一项重要任务，该协议借助提前制备好的纠缠资源在两个空间上分离的通信节点（仅允许经典信道）之间传输量子信息。在本教程中，我们将首先简要回顾一下量子隐形传态协议，并使用量桨进行模拟。然后，我们将介绍如何使用 LOCCNet 学习出一个量子隐形传态协议。

## 量子隐形传态协议

量子隐形传态协议最初由 C. H. Bennett 等人在 1993 年提出 [1]，并在 1997 年通过基于光子的实验平台进行了验证 [2-3]。其主要工作流程如下图所示。传输过程中需要 2 个通信节点或参与方，即 Alice 和 Bob。简单起见，我们仅考虑传输一个单量子比特的量子态 $|\psi\rangle_C$，这样整个系统总共需要 3 个量子比特，包括提前共享给 Alice 和 Bob 的最大纠缠态 $|\Phi^+\rangle_{AB} = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$。 Alice 持有量子比特 A 和 C， Bob 持有量子比特 B。**注释：量子隐形传态协议仅传输量子信息，而非在物理上直接传输量子比特。**


<center><img src="figures/teleportation-fig-circuit.jpg" height="400" width="600"></center>
<div style="text-align:center">图 1：量子隐形传态: 传输量子态 $|\psi\rangle$ 从 Alice 到 Bob </div>



**步骤 I：** 一开始，整个量子系统的状态可以描述为

$$
\lvert\varphi_{0}\rangle 
= \lvert\psi\rangle_{C}\otimes \lvert\Phi^+\rangle_{AB} 
= \frac{1}{\sqrt{2}}\big[\alpha\lvert0\rangle(\lvert00\rangle + \lvert11\rangle)+\beta\lvert1\rangle(\lvert00\rangle + \lvert11\rangle)\big],
\tag{1}
$$

 其中 Alice 要传输的量子态是 $|\psi\rangle_C = \alpha|0\rangle_C + \beta|1\rangle_C$ 并且 $\alpha, \beta \in \mathbb{C}$。

**步骤 II：**  Alice 在她持有的两个量子比特上作用 CNOT 门，得到作用后的量子态为

$$
|\varphi_1\rangle  
= \frac{1}{\sqrt{2}}\big[\alpha\lvert0\rangle(\lvert00\rangle + \lvert11\rangle)+\beta\lvert1\rangle(\lvert10\rangle + \lvert01\rangle)\big],
\tag{2}
$$

**步骤 III：** 接着 Alice 在她持有的量子比特 $C$ 上作用 Hadamard 门，使得整个系统的状态变为 $|\varphi_2\rangle$

$$
|\varphi_2\rangle = \frac{1}{2}\big[\alpha(\lvert0\rangle + \lvert1\rangle)(\lvert00\rangle + \lvert11\rangle)+\beta(\lvert0\rangle - \lvert1\rangle)(\lvert10\rangle + \lvert01\rangle)\big],
\tag{3}
$$

为了更好地观察后续的测量结果，不妨将上述态重新写为

$$
\lvert\varphi_{2}\rangle = \frac{1}{2}\big[\lvert00\rangle(\alpha\lvert0\rangle + \beta\lvert1\rangle) + \lvert01\rangle(\alpha\lvert1\rangle + \beta\lvert0\rangle) + \lvert10\rangle(\alpha\lvert0\rangle - \beta\lvert1\rangle) + \lvert11\rangle(\alpha\lvert1\rangle - \beta\lvert0\rangle)\big].
\tag{4}
$$

**步骤 IV：**  Alice 在计算基（computational basis）$\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$ 上测量她的两个量子比特并将结果 $m_1m_2$ 通过经典信道发送给 Bob 。 总共有 4 种不同的可能性： $m_1m_2 \in \{ 00, 01,10, 11\}$。 然后， Bob 根据收到的消息在其量子比特 $B$ 上进行对应的操作。

- 如果测量结果为 $m_1m_2 = 00$，则 Bob 的态将为 $\alpha\lvert0\rangle + \beta\lvert1\rangle$。 无需任何操作即可完成传输。
- 如果测量结果为 $ m_1m_2 = 01 $，则 Bob 的态将为 $\alpha\lvert1\rangle + \beta\lvert0\rangle$。  Bob 需要在其量子比特上作用 $ X $ 门。
- 如果测量结果为 $ m_1m_2 = 10 $，则 Bob 的态将为 $\alpha\lvert0\rangle - \beta\lvert1\rangle$。  Bob 需要在其量子比特上作用 $ Z $ 门。
- 如果测量结果为 $ m_1m_2 = 11 $，则 Bob 的态将为 $\alpha\lvert1\rangle - \beta\lvert0\rangle$。 Bob 需要在其量子比特上执行 $ X $ 门操作，然后执行 $ Z $ 门操作。

在下一节中，我们将介绍如何使用量桨模拟量子隐形传态协议。

## Paddle Quantum 代码实现

首先，我们需要导入所有依赖包：

In [1]:
import numpy as np
import paddle.fluid as fluid
from paddle_quantum.locc import LoccNet
from paddle.complex import matmul, trace
from paddle_quantum.utils import state_fidelity
from paddle_quantum.state import bell_state, isotropic_state, density_op_random

初始化整个量子系统，然后定义量子电路和隐形传态协议。

In [2]:
class LOCC(LoccNet):
    def __init__(self):
        super(LOCC, self).__init__()
        
        # 初始化 LOCCNet
        self.parties = list()
        
        # 添加第一个参与方 Alice
        # 第一个参数 2 代表着 Alice 手里有几个量子比特
        # 第二个参数代表着参与方的名字
        self.add_new_party(2, party_name="Alice")
        
        # 添加第二个参与方 Bob
        # 第一个参数 1 代表着 Bob 手里有几个量子比特
        # 第二个参数代表着参与方的名字
        self.add_new_party(1, party_name="Bob")
        

        # 准备一个贝尔态
        _state = fluid.dygraph.to_variable(bell_state(2))
#         _state = fluid.dygraph.to_variable(isotropic_state(2, 0.8))

        
        # 随机制备传输用的纯态 （rank =1）
        random_state = density_op_random(n=1, real_or_complex=2, rank=1)
        self.state_C = fluid.dygraph.to_variable(random_state)
        
        # 通过分配上述制备好的量子态初始化整个量子系统
        # 这里 ("Alice", 0) 即表示量子比特 C
        # 这里 ("Alice", 1) 即表示量子比特 A
        # 这里 ("Bob", 0) 即表示量子比特 B
#         print('提前分配好的纠缠态为:\n', _state.numpy())
        self.set_init_status(self.state_C, [("Alice", 0)])
        self.set_init_status(_state, [("Alice", 1), ("Bob", 0)])


    def teleportation(self):
        status = self.init_status
        
        # 设置 Alice 的本地操作
        cirA = self.create_ansatz("Alice")
        cirA.cnot([0, 1])
        cirA.h(0)
        
        # 运行上述电路
        status = cirA.run(status)
        
        # Alice 在计算基上测量她所持有的两个量子比特 C 还有 A
        # 得到并记录四种结果 00，01，10，11
        status_A = self.measure(status, [("Alice", 0), ("Alice", 1)], ["00", "01", "10", "11"])
        
        # 用于记录平均保真度
        fid_list = []

        # Bob 根据 Alice 的测量结果选择不同的门作用在自己的量子比特上
        for i, s in enumerate(status_A):
            
            # 判断语句根据 Alice 的测量结果，进行不同操作 
            if status_A[i].measured_result == '00':
                
                # 创建 Bob 的本地操作
                cirB = self.create_ansatz("Bob")
                
                # 执行电路
                status_B = cirB.run(s) 
                
                # 仅保留 Bob 的量子比特 B
                status_fin = self.partial_state(status_B, [("Bob", 0)])

                # 计算初始态和传输后态之间的保真度
                fid = state_fidelity(self.state_C.numpy(), status_fin.state.numpy())**2
                fid_list.append(fid * status_fin.prob.numpy()[0])
                
            # 以下操作类似
            elif status_A[i].measured_result == '01':
                cirB = self.create_ansatz("Bob")
                cirB.x(0)
                status_B = cirB.run(s)
                status_fin = self.partial_state(status_B, [("Bob", 0)])
                fid = state_fidelity(self.state_C.numpy(), status_fin.state.numpy())**2
                fid_list.append(fid * status_fin.prob.numpy()[0])
            elif status_A[i].measured_result == '10':
                cirB = self.create_ansatz("Bob")
                cirB.z(0)
                status_B = cirB.run(s)
                status_fin = self.partial_state(status_B, [("Bob", 0)])
                fid = state_fidelity(self.state_C.numpy(), status_fin.state.numpy())**2
                fid_list.append(fid * status_fin.prob.numpy()[0])
            elif status_A[i].measured_result == '11':
                cirB = self.create_ansatz("Bob")
                cirB.x(0)
                cirB.z(0)
                status_B = cirB.run(s)
                status_fin = self.partial_state(status_B, [("Bob", 0)])
                fid = state_fidelity(self.state_C.numpy(), status_fin.state.numpy())**2
                fid_list.append(fid * status_fin.prob.numpy()[0])
        fid_avg = sum(fid_list)
        
        return fid_avg

然后，我们随机生成 200 个量子纯态，并使用态保真度 $ F $ 来衡量传输协议好坏，其中

$$
F(\rho,\sigma) \equiv \text{tr}\big( \sqrt{\sqrt{\rho}\sigma \sqrt{\rho}} \big)^2.
\tag{5}
$$

In [5]:
with fluid.dygraph.guard():
    
    np.random.seed(999)     # 固定生成传输态的随机种子
    num_state = 200         # 设置随机态的生成数量
    list_fid = []           # 用于记录保真度
    
    # 开始采样 
    for idx in range(num_state):
        list_fid.append(LOCC().teleportation())

    print('平均保真度 =', np.around(sum(list_fid)/len(list_fid), 4), ', 标准差 =', np.std(list_fid))

平均保真度 = 1.0 , 标准差 = 6.429150606796583e-09


**注释：** 我们要指出的是，该协议的有效性取决于提前分配好的贝尔态的品质。 感兴趣的读者可以将纠缠态从 `bell_state(2)` 更改为 `isotropic_state(2, p)`，并测试观察在贝尔态中出现的量子噪声将如何影响该协议的性能。 其中 isotropic 态的定义如下，

$$
\rho_{\text{iso}}(p) = p\lvert\Phi^+\rangle \langle\Phi^+\rvert + (1-p)\frac{I}{4}, \quad p \in [0,1]
\tag{6}
$$

## 使用 LOCCNet 学习量子隐形传态

### 训练一个自定义的 LOCC 协议

一般的 LOCC 协议可以通过经典通信的回合数 $ r $ 来进行分类。 原始的量子隐形传态协议是单轮通讯协议（$ r = 1 $）。 为简单起见，我们也将通信回合数限制为单轮。与原始的协议不同，我们将使用参数化量子电路（PQC）来把 Bob 作用在量子比特上的固定门 $U\in\{X,Z\}$ 替换成布洛赫球面上的一个广义旋转门 $ U_3 $， 其定义为

$$
U_3(\theta, \phi, \varphi) =
\begin{bmatrix}
\cos(\frac{\theta}{2})           & -e^{i\varphi}\sin(\frac{\theta}{2})\\
e^{i\phi}\sin(\frac{\theta}{2})  & e^{i(\phi+\varphi)} \cos(\frac{\theta}{2})
\end{bmatrix}.
\tag{7}
$$

这将为我们带来更强大的搜索能力，来寻找更多使用场景下的 LOCC 协议。类似地，我们将 Alice 的本地操作更改为更通用的参数化量子电路，这里具体使用的是一个内置的两量子比特通用门 `universal_2_qubit_gate(theta, which_qubit=[0,1])` [4]。基于以上条件，我们训练一个自定义的 LOCC 协议流程如下：

1. Alice 对她所持有的两个量子比特作用两量子比特通用门。
2. 然后 Alice 在计算基上测量她的两个量子比特，并通过经典信道与 Bob 交流。
3. 共计有 4 种可能的测量结果：$m_1m_2 \in \{00, 01, 10, 11\}$。 Bob 需要根据这些测量结果采取不同的本地操作。在 Bob 进行操作后，记其量子态为 $\lvert \psi\rangle_{B}$。
4. 计算 $\lvert \psi\rangle_{B}$ 与 $\lvert\psi\rangle_C$（纯态）之间的量子态重叠（state overlap）并记为 $ O $。 由于 LOCCNet 框架目前仅支持密度矩阵形式，因此我们必须将它们重写为 $\rho_{B} = |\psi\rangle\langle\psi|_B$ 和 $\rho_{C} = |\psi\rangle\langle\psi|_C$。然后可以得到 $O = \text{Tr}(\rho_C\rho_{B})$。对于纯态，此距离度量就是保真度 (fidelity)。
5. 将损失函数设置为 4 种可能测量结果的累加，即 $L = \sum_{m_1m_2} \big(1-\text{Tr}(\rho_C\rho_{B})\big)$，并使用基于梯度的优化方法更新 Alice 和 Bob 本地操作中的参数，从而使得损失函数最小化。
6. 重复步骤 1-5，直到损失函数收敛。
7. 生成一组随机的态 $\{\lvert\psi_C\rangle\}$，并以平均保真度对训练出的传输协议进行基准测试。



<center><img src="figures/teleportation-fig-LOCCNet.png" height="400" width="600"></center>
<div style="text-align:center">图 2： 用 LOCCNet 学习量子隐形传态协议 </div>



**注释：** 为了确保训练出的 LOCC 协议对所有态均有效，我们将训练集设为 4 个线性独立态，即 $\{|0\rangle\langle 0|,|1\rangle\langle 1|,|+\rangle\langle +|,|+\rangle\langle +|_y\}$ 。其中 $|+\rangle, |+\rangle_y$ 分别为泡利 $X,Y$ 矩阵的正特征值对应的特征向量。 该训练集在密度矩阵的表示下为：

$$
\rho_0 = \left[\begin{array}{ccc}
1 & 0\\
0 & 0
\end{array}\right], 
\rho_1 = \left[\begin{array}{ccc}
0 & 0\\
0 & 1
\end{array}\right], 
\rho_2 = \left[\begin{array}{ccc}
0.5 & 0.5\\
0.5 & 0.5
\end{array}\right], 
\rho_3 = \left[\begin{array}{ccc}
0.5 & -0.5 i\\
0.5i & 0.5
\end{array}\right]. 
\tag{8}
$$

任何一个单量子比特量子态都可以写为上述 4 个态的线性组合。

In [ ]:
class LOCC_Train(LoccNet):
    def __init__(self):
        super(LOCC_Train, self).__init__()
        
        # 初始化 LOCCNet
        self.parties = list()
        
        # 添加第一个参与方 Alice
        # 第一个参数 2 代表着 Alice 手里有几个量子比特
        # 第二个参数代表着参与方的名字
        self.add_new_party(2, party_name="Alice")
        
        # 添加第二个参与方 Bob
        # 第一个参数 1 代表着 Bob 手里有几个量子比特
        # 第二个参数代表着参与方的名字
        self.add_new_party(1, party_name="Bob")
        

        # 准备一个贝尔态
        _state = fluid.dygraph.to_variable(bell_state(2))
#         _state = fluid.dygraph.to_variable(isotropic_state(2, 0.8))


        # 设置 Alice 本地操作的参数
        self.theta_A = self.create_parameter(shape=[15], attr=fluid.initializer.Uniform(low=0.0, high=2 * np.pi, 
                                                                                        seed=SEED), dtype="float64")
        # 设置 Bob 本地操作的参数
        self.theta_B = self.create_parameter(shape=[4, 3], attr=fluid.initializer.Uniform(low=0.0, high=2 * np.pi, 
                                                                                          seed=SEED), dtype="float64")

        # 训练集: 4 个线性独立态
        _state0 = fluid.dygraph.to_variable(np.array([[1, 0], [0, 0]], dtype=np.complex128))
        _state1 = fluid.dygraph.to_variable(np.array([[0, 0], [0, 1]], dtype=np.complex128))
        _state2 = fluid.dygraph.to_variable(np.array([[0.5, 0.5], [0.5, 0.5]], dtype=np.complex128))
        _state3 = fluid.dygraph.to_variable(np.array([[0.5, -0.5j], [0.5j, 0.5]], dtype=np.complex128))
        self.init_states = [_state0, _state1, _state2, _state3]

        # 通过分配上述制备好的量子态初始化整个量子系统
        self.set_init_status(_state, [("Alice", 1), ("Bob", 0)])
        self.set_init_status(_state0, [("Alice", 0)])

    def LOCCNet(self):
        
        # 定义训练过程
        loss = 0
        temp_state = self.init_status
        
        # 开始训练
        for init_state in self.init_states:
            
            # 重置 Alice 持有的量子比特 C 至训练集中的量子态
            status = self.reset_state(temp_state, init_state, [("Alice", 0)])

            # 定义 Alice 的本地操作
            cirA = self.create_ansatz("Alice")
            cirA.universal_2_qubit_gate(self.theta_A, [0, 1])

            # 执行 Alice 的电路
            status = cirA.run(status)
            
             # 测量得到四个可能的结果
            status_A = self.measure(status, [("Alice", 0), ("Alice", 1)], ["00", "01", "10", "11"])
            
            # Bob 根据测量结果选择不同的门作用在自己的量子比特上
            for i, s in enumerate(status_A):
                
                # 定义 Bob 的本地操作
                cirB = self.create_ansatz("Bob")
                
                # 作用单量子比特通用门
                cirB.u3(*self.theta_B[i], 0)
                
                # 执行 Bob 的电路
                status_B = cirB.run(s)
                
                # 仅留下 Bob 的量子比特 B
                status_fin = self.partial_state(status_B, [("Bob", 0)])
                
                # 将所有的测量结果的损失函数进行累加
                loss += 1 - trace(matmul(init_state, status_fin.state)).real[0]
        
        return loss

    # 存储训练结束后的最优参数
    def save_module(self):
        theta = np.array([self.theta_A.numpy(), self.theta_B.numpy()])
        np.save('parameters/QT_LOCCNet', theta)

In [ ]:
ITR = 150   # 设置优化循环次数
LR = 0.2    # 设置学习速率
SEED = 999  # 固定本地操作中参数的初始化随机数种子

# 初始化动态图模式
with fluid.dygraph.guard():

    net = LOCC_Train()
    
    # 选择 Adam 优化器
    opt = fluid.optimizer.AdamOptimizer(learning_rate=LR, parameter_list= net.parameters())
    
    # 记录损失
    loss_list = []
    
    # 开始优化循环
    for itr in range(ITR):

        # 向前传播计算损失函数
        loss = net.LOCCNet()
        
        # 反向传播优化损失函数
        loss.backward()
        opt.minimize(loss)
        
        # 清除梯度
        net.clear_gradients()
        
        # 记录训练结果
        loss_list.append(loss.numpy()[0])
        if itr % 10 == 0:
            print("itr " + str(itr) + ":", loss.numpy()[0])
            
    # 保存参数
    net.save_module()

### 基准测试

如果想要更快捷的感受效果，我们还提供**加载预先训练好的电路参数**并直接测试性能。

In [6]:
class LOCC_Test(LoccNet):
    def __init__(self):
        super(LOCC_Test, self).__init__()
        
        self.parties = list()
        self.add_new_party(2, party_name="Alice")
        self.add_new_party(1, party_name="Bob")
        
        self.theta_A = fluid.dygraph.to_variable(para[0])
        self.theta_B = fluid.dygraph.to_variable(para[1])
        
        _state = fluid.dygraph.to_variable(bell_state(2))
        random_state = density_op_random(n=1)
        self._state0 = fluid.dygraph.to_variable(random_state)
        self.set_init_status(_state, [("Alice", 1), ("Bob", 0)])
        self.set_init_status(self._state0, [("Alice", 0)])
        

    def benchmark(self):
        input_state = self.init_status
        cirA = self.create_ansatz("Alice")
        cirA.universal_2_qubit_gate(self.theta_A, [0, 1])

        status = cirA.run(input_state)
        status_A = self.measure(status, [("Alice", 0), ("Alice", 1)], ["00", "01", "10", "11"])
        fid_list = []

        for i, s in enumerate(status_A):
            cirB = self.create_ansatz("Bob")
            cirB.u3(*self.theta_B[i], 0)
            status_B = cirB.run(s)
            status_fin = self.partial_state(status_B, [("Bob", 0)])
            fid = state_fidelity(self._state0.numpy(), status_fin.state.numpy())**2
            fid_list.append(fid*status_fin.prob.numpy()[0])
        fid_ave = sum(fid_list)
        
        return fid_ave

In [8]:
with fluid.dygraph.guard():

    # 加载预先训练的电路参数
    para = np.load('parameters/QT_LOCCNet_Trained.npy', allow_pickle=True)
    np.random.seed(999)     # 固定生成传输态的随机种子
    num_state = 200         # 设置随机态的生成数量
    list_fid = []           # 用于记录保真度
    
    # 采样
    for idx in range(num_state):
        list_fid.append(LOCC_Test().benchmark())

    print('平均保真度 =', np.around(sum(list_fid)/len(list_fid), 4), ', 标准差 =', np.std(list_fid))

平均保真度 = 1.0 , 标准差 = 5.695904177817817e-07


## 结论

从以上数值实验的结果可以看出，基于 LOCCNet 我们成功地学习出了量子隐形传态协议。 最初的隐形传态协议旨在传输单量子比特量子态，无法直接推广到多量子比特的情形。 相比之下，LOCCNet 为寻找多量子比特情形下的隐形传态协议提供了可能, 更是可以训练得到更一般的通过量子纠缠和 LOCC 实现不同节点间量子信道的模拟。 此外，感兴趣的读者可以尝试各类量子噪声会如何影响 LOCCNet 训练的效果。 

---

## 参考文献

[1] Bennett, Charles H., et al. "Teleporting an unknown quantum state via dual classical and Einstein-Podolsky-Rosen channels." [Physical Review Letters 70.13 (1993): 1895.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.70.1895)

[2] Boschi, Danilo, et al. "Experimental realization of teleporting an unknown pure quantum state via dual classical and Einstein-Podolsky-Rosen channels." [Physical Review Letters 80.6 (1998): 1121.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.80.1121)

[3] Bouwmeester, Dik, et al. "Experimental quantum teleportation." [Nature 390.6660 (1997): 575-579.](https://www.nature.com/articles/37539)

[4] Vidal, Guifre, and Christopher M. Dawson. "Universal quantum circuit for two-qubit transformations with three controlled-NOT gates." [Physical Review A 69.1 (2004): 010301.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.69.010301)